In [ ]:
import torch
import os
from dotenv import load_dotenv

from huggingface_hub import login
from transformers import BitsAndBytesConfig

from llama_index.core import Document
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import Settings
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.response.notebook_utils import display_response

import logging
import sys
import gradio as gr

In [ ]:
# Load environment variables from the .env file
load_dotenv()

# Set the HF_TOKEN environment variable from .env or fallback to a default key if not present
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

# Retrieve the token from the environment variable
# If it's not set in the environment, it will fall back to the default or user-provided value
hf_token = os.environ['HF_TOKEN']

# Log in to Hugging Face API using the retrieved token
login(hf_token, add_to_git_credential=True)

# Print a success message after login
print("Successfully logged in to Hugging Face!")

In [4]:
# Function to read a text file
def read_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()  # Read the content of the file
    return content

# Specify the path of your text file
txt_file_path = 'reviews_document.txt'

# Read the file content
file_content = read_txt_file(txt_file_path)

# Create a Document object from the content
documents = Document(text=file_content)

In [5]:
# Function to convert a list of messages to a formatted prompt for the model
def messages_to_prompt(messages):
    prompt = ""  # Initialize the prompt string
    system_found = False  # Flag to check if a system message exists

    for message in messages:
        # If the message role is "system", format it as a system message
        if message.role == "system":
            prompt += f"<|system|>\n{message.content}<|end|>\n"
            system_found = True
        # If the message role is "user", format it as a user message
        elif message.role == "user":
            prompt += f"<|user|>\n{message.content}<|end|>\n"
        # If the message role is "assistant", format it as an assistant message
        elif message.role == "assistant":
            prompt += f"<|assistant|>\n{message.content}<|end|>\n"
        # If the role is not recognized, assume it's a user message
        else:
            prompt += f"<|user|>\n{message.content}<|end|>\n"

    # Add a trailing assistant prompt
    prompt += "<|assistant|>\n"

    # If no system message was found, prepend a default system message
    if not system_found:
        prompt = (
            "<|system|>\nYou are a helpful AI assistant for analyzing games with reviews you achieve. Don't make up, just analyze the data and answer. If the game in the prompt is not in the data, say 'I don't know about this game.'<|end|>\n" + prompt
        )

    return prompt


In [ ]:
# Configuration for loading the model with 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)


# Initialize the LLM with specified settings
llm = HuggingFaceLLM(
    model_name="microsoft/Phi-3.5-mini-instruct",  # Model name
    model_kwargs={
        "trust_remote_code": True,
        "quantization_config": quantization_config  # Apply quantization settings
    },
    generate_kwargs={"do_sample": True, "temperature": 0.1}, # Text generation settings
    tokenizer_name="microsoft/Phi-3.5-mini-instruct",
    query_wrapper_prompt=(
        "<|system|>\n"
        "You are a helpful AI assistant for analyzing games with reviews you achieve. Dont make up just analyze the data and answer. If the game in prompt is not in the data, say I don't know about this game.<|end|>\n"
        "<|user|>\n"
        "{query_str}<|end|>\n"
        "<|assistant|>\n"
    ),
    messages_to_prompt=messages_to_prompt,
    is_chat_model=True,  # Use chat-based model
    device_map="auto"
)

In [ ]:
# Set the LLM model in settings
Settings.llm = llm

# Set the embedding model in settings
Settings.embed_model = HuggingFaceEmbedding(
    model_name="all-MiniLM-L6-v2"
)

In [8]:
# Initialize sentence splitter with chunk size and overlap settings
node_parser = SentenceSplitter(
    chunk_size=100,
    chunk_overlap=20
)

# Create a vector store index from the documents using the embedding model
vector_index = VectorStoreIndex.from_documents(
    [documents],
    embed_model=Settings.embed_model,
    node_parser=node_parser
)

In [9]:
# Configure logging to output to stdout with INFO level
logging.basicConfig(stream=sys.stdout, level=logging.INFO)

# Add a stream handler to the logger to output logs to stdout
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [10]:
# Create a query engine from the vector index with compact response mode
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("What do you know about Dead by daylight? Do you recommend this game?")

# Display the response from the query engine
display_response(response)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


You are not running the flash-attention implementation, expect numerical differences.


**`Final Response:`** Dead by Daylight is a game that offers a thrilling experience, especially when played with friends. It's described as a great game for playing with friends, with the potential to be fun even when played solo. The game is noted for its intense gameplay, where players can either be a survivor or a killer, each with their unique roles and objectives.

The game has been praised for its graphics, gameplay, and the sense of suspense it builds. It's also mentioned that the game can be challenging, but it's recommended for those who enjoy horror games and are willing to put in the effort to master it.

Many players express their love for the game, highlighting its enjoyable aspects, such as the fun of hiding and seeking, the excitement of being hunted, and the thrill of the gameplay. Some players also mention the game's potential for fun with friends, and the enjoyment they get from the game's community, despite some toxicity.

Based on the context, it is clear that many players highly recommend Dead by Daylight, citing its fun gameplay, enjoyable aspects, and the satisfaction of playing with friends. However,

In [11]:
# Create a query engine from the vector index with compact refine mode
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("What do you know about Dead by daylight? Do you recommend this game?")

display_response(response)

**`Final Response:`** Dead by daylight is a popular horror-themed multiplayer game that emphasizes the thrill of being hunted or the hunter, offering a unique and engaging experience. It's particularly enjoyable when played with friends, as it allows for social interaction and teamwork. Despite some players pointing out the need for bug fixes and a desire for more game modes, the game has garnered positive feedback for its graphics, gameplay, and the excitement it brings to players. Many have expressed their love for the game and have recommended it to others, suggesting that it's a great choice for those who appreciate horror games and value the social aspect of gaming. The game seems to have a strong community, with players often sharing their experiences and strategies, which enhances the overall enjoyment. If you're looking for a game that combines horror elements with social play, Dead by daylight could be a fun option to consider.

In [16]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("Based on reviews you get which game do you think as a best game?")

display_response(response)

**`Final Response:`** Based on the context provided, many reviewers express strong positive sentiments towards PUBG, highlighting its fun gameplay, constant adrenaline, better graphics compared to other games like Fortnite, and its potential as a great alternative to H1Z1. Despite some issues with hackers, servers, and the need for improvements, the overall tone suggests that PUBG is highly regarded and considered a top choice among the reviewed games. Therefore, PUBG is suggested as the best game in this context.

In [17]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("Based on reviews you get which game do you think as a best game?")

display_response(response)

**`Final Response:`** Based on the reviews, PUBG is considered the best game. It's highly praised for its fun gameplay, constant adrenaline rush, and overall greatness. Despite some issues, it's frequently recommended by many reviewers. Its popularity is also indicated by its high ranking on Twitch. The positive feedback and high recommendation rate suggest that PUBG stands out as the top choice among the games mentioned.

In [18]:
query_engine = vector_index.as_query_engine(response_mode="compact")

# Chatbot function to process messages and maintain history
def chatbot(message, history):
    try:
        # Execute the query to get a response
        response = query_engine.query(message)
        return str(response)
    except Exception as e:
        return f"An error occurred: {str(e)}"

In [19]:
# Create the Gradio interface for the chatbot
iface = gr.ChatInterface(
    fn=chatbot,
    title="Game Reviews Chatbot",
    description="Ask me about games and their reviews!"
)

# Launch the application
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://550aa2905f3d7ce00b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
